In [1]:
# Import the libraries (Already installed)
from dotenv import load_dotenv
import os
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

In [2]:
# Load .env variables into environment
load_dotenv()

# Access the Groq key
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

![img](RAG_basic.png)

---

# Step 1. Indexing
### Step 1.1: Document Ingestion

In [3]:
VIDEO_ID = "Gfr50f6ZBvo"

In [4]:
try:
    ytt_api = YouTubeTranscriptApi()
    fetched = ytt_api.fetch(VIDEO_ID, languages=["en"])  # FetchedTranscript
    transcript_list = fetched.to_raw_data()              # [{text,start,duration}, ...]
    transcript = " ".join(x["text"] for x in transcript_list).strip()
except TranscriptsDisabled:
    print("No captions available for this video.")
    transcript = ""
except Exception as e:
    print(f"Failed to fetch transcript: {e}")
    transcript = ""

if not transcript:
    raise SystemExit("Cannot proceed: no transcript retrieved.")

In [5]:
# Peek into the downloaded transcript
print(transcript_list[:10])

[{'text': 'the following is a conversation with', 'start': 0.08, 'duration': 3.44}, {'text': 'demus hasabis', 'start': 1.76, 'duration': 4.96}, {'text': 'ceo and co-founder of deepmind', 'start': 3.52, 'duration': 5.119}, {'text': 'a company that has published and builds', 'start': 6.72, 'duration': 4.48}, {'text': 'some of the most incredible artificial', 'start': 8.639, 'duration': 4.561}, {'text': 'intelligence systems in the history of', 'start': 11.2, 'duration': 4.8}, {'text': 'computing including alfred zero that', 'start': 13.2, 'duration': 3.68}, {'text': 'learned', 'start': 16.0, 'duration': 2.96}, {'text': 'all by itself to play the game of gold', 'start': 16.88, 'duration': 4.559}, {'text': 'better than any human in the world and', 'start': 18.96, 'duration': 5.6}]


### Step 1.2: Text Splitting

In [6]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [7]:
docs = splitter.create_documents([transcript])

In [8]:
# Peek into a chunk
print(docs[1])

page_content='out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get good enough to interview you well i'll be impressed if if you were i'd be impressed by myself if you were i don't think we're quite up to that yet but uh maybe you're from the future lex if you did would you tell me is that is that a good thing to tell a language model that's tasked with interviewing that it is in fact um ai maybe we're in a kind of meta turing test uh probably probably it would be a good idea not to tell you so it doesn't change your behavior right this is a kind of heisenberg uncertainty principle situation if i told you you behave differently yeah maybe that's what's happening with us of course this is a benchmark from the future where they replay 2022 as a year before ais were good enough yet and now we want to see is it going to pass exactly if i was such a program wo

In [9]:
# Size
print(f"Size of chunks/docs: {len(docs)}")

Size of chunks/docs: 168


### Step 1.3: Embeddings Generation

In [10]:
embeddings = FastEmbedEmbeddings()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

### Step 1.4: Storing in Vector Store

In [11]:
vector_store = FAISS.from_documents(docs, embeddings)

In [12]:
print(vector_store.index_to_docstore_id)

{0: '3f3c9e70-8cf4-4699-ade8-15e30afe1f65', 1: 'f94330a2-d3fe-4b2d-bb74-b3e651dc2c93', 2: '6db4b4f5-3237-46ce-8e7a-191e4be4a710', 3: '126f7dc1-377b-4072-ac63-f8ca40344086', 4: 'f5c17d8f-23b1-42e5-ae32-0adc4931348b', 5: '6a554f52-fd46-4def-9992-ccb29f8555bd', 6: '4c083c4c-af57-4f41-99fc-76fac1c4453f', 7: '32c66f49-0cbf-45fa-bf0d-340899a6a88f', 8: '82dbce1f-b2be-494c-8b17-92a49ee5374f', 9: 'ba1a29ff-cac0-482d-886b-bf88c5a59228', 10: '53172e08-935b-478e-b9fd-95462c26d57e', 11: '810a5023-7d5e-4513-a732-4668136c2bf7', 12: 'b5791862-e569-42d3-87fb-a081df3b2a52', 13: 'c564de34-d451-4f17-aa98-8f17156c06dc', 14: '1e6acf41-b1ed-46dd-aba6-fa0cdc8f291e', 15: 'e885da91-aefe-4476-819b-e096b0cbb626', 16: '6a7cc512-2575-4e5e-890e-68835e1f5750', 17: '57187b24-2dc4-4708-af78-bd25e5b41895', 18: 'd404604f-faf4-42ba-b392-88575a997b60', 19: 'e10d9965-abca-432c-bbe2-a4548ecab7b5', 20: 'dff95eb5-a950-4355-99d8-a1fa51c04f3f', 21: '11af1158-dc47-46ea-bf60-6bc6d5c9a904', 22: '89d099b1-c2ee-4605-922f-346e7cf0d754

In [13]:
try_id = vector_store.index_to_docstore_id[0]
print(f"Accessing the chunk at ID: {try_id}")

# Peeking into a chunk via its ID:
vector_store.get_by_ids([try_id])

Accessing the chunk at ID: 3f3c9e70-8cf4-4699-ade8-15e30afe1f65


[Document(id='3f3c9e70-8cf4-4699-ade8-15e30afe1f65', metadata={}, page_content="the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal qu

# Step 2. Retrieval

In [14]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [15]:
print(retriever)

tags=['FAISS', 'FastEmbedEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002156D6C9410> search_kwargs={'k': 4}


In [16]:
retriever.invoke('What is DeepMind?')

[Document(id='86c1076c-b5e0-4c57-97b4-9b3c1704406c', metadata={}, page_content="and how it works this is tough to uh ask you this question because you probably will say it's everything but let's let's try let's try to think to this because you're in a very interesting position where deepmind is the place of some of the most uh brilliant ideas in the history of ai but it's also a place of brilliant engineering so how much of solving intelligence this big goal for deepmind how much of it is science how much is engineering so how much is the algorithms how much is the data how much is the hardware compute infrastructure how much is it the software computer infrastructure yeah um what else is there how much is the human infrastructure and like just the humans interact in certain kinds of ways in all the space of all those ideas how much does maybe like philosophy how much what's the key if um uh if if you were to sort of look back like if we go forward 200 years look back what was the key 

# Step 3. Augmentation

In [17]:
llm = ChatGroq(model="llama-3.1-8b-instant", 
               api_key = GROQ_API_KEY, 
               temperature=0.2)

In [18]:
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are a concise, helpful assistant. Use ONLY the context to answer.\n\n"
        "Context:\n{context}\n\n"
        "Question: {question}\n\n"
        "If the answer isn't in the context, say you can't find it."
    ),
)

In [19]:
question  = "Has the topic of nuclear fusion been discussed in this video? If yes, then what was discussed?"
retrieved_docs = retriever.invoke(question)

In [20]:
retrieved_docs

[Document(id='e7791877-84a6-47b7-ad1f-a8879c2219fb', metadata={}, page_content="so we with this problem and we published it in a nature paper last year uh we held the fusion that we held the plasma in specific shapes so actually it's almost like carving the plasma into different shapes and control and hold it there for the record amount of time so um so that's one of the problems of of fusion sort of um solved so i have a controller that's able to no matter the shape uh contain it continue yeah contain it and hold it in structure and there's different shapes that are better for for the energy productions called droplets and and and so on so um so that was huge and now we're looking we're talking to lots of fusion startups to see what's the next problem we can tackle uh in the fusion area so another fascinating place in a paper title pushing the frontiers of density functionals by solving the fractional electron problem so you're taking on modeling and simulating the quantum mechanical 

In [21]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"so we with this problem and we published it in a nature paper last year uh we held the fusion that we held the plasma in specific shapes so actually it's almost like carving the plasma into different shapes and control and hold it there for the record amount of time so um so that's one of the problems of of fusion sort of um solved so i have a controller that's able to no matter the shape uh contain it continue yeah contain it and hold it in structure and there's different shapes that are better for for the energy productions called droplets and and and so on so um so that was huge and now we're looking we're talking to lots of fusion startups to see what's the next problem we can tackle uh in the fusion area so another fascinating place in a paper title pushing the frontiers of density functionals by solving the fractional electron problem so you're taking on modeling and simulating the quantum mechanical behavior of electrons yes um can you explain this work and can ai model and\n\n

In [22]:
final_prompt = prompt.invoke({"context": context_text, "question": question})
print(final_prompt)

text="You are a concise, helpful assistant. Use ONLY the context to answer.\n\nContext:\nso we with this problem and we published it in a nature paper last year uh we held the fusion that we held the plasma in specific shapes so actually it's almost like carving the plasma into different shapes and control and hold it there for the record amount of time so um so that's one of the problems of of fusion sort of um solved so i have a controller that's able to no matter the shape uh contain it continue yeah contain it and hold it in structure and there's different shapes that are better for for the energy productions called droplets and and and so on so um so that was huge and now we're looking we're talking to lots of fusion startups to see what's the next problem we can tackle uh in the fusion area so another fascinating place in a paper title pushing the frontiers of density functionals by solving the fractional electron problem so you're taking on modeling and simulating the quantum me

# Step 4: Generation

In [23]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, the topic of nuclear fusion has been discussed in this video. 

The discussion includes:

1. The problem of fusion and how it was solved by holding the plasma in specific shapes for a record amount of time.
2. A controller that can contain and hold the plasma in different shapes, including droplets, which are better for energy production.
3. Collaboration with EPFL in Switzerland to use their test reactor and apply AI methods to solve the bottleneck problems in fusion.
4. A paper on magnetic control of tokamak plasmas using deep reinforcement learning.
5. The potential for AI to help accelerate and solve the challenges in fusion, which is one of the biggest areas in energy and climate.

The speaker also mentions that they are looking to tackle the next problem in fusion and are talking to fusion startups to identify the next challenge to solve.


---

# Building a Chain with the help of LangChain

In [24]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [25]:
parallel_chain = RunnableParallel({
    "context": retriever | RunnableLambda(format_docs),
    "question": RunnablePassthrough(),
})

In [26]:
parallel_chain.invoke('Who is Demis?')

{'context': "the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get

In [27]:
parser = StrOutputParser()

In [28]:
main_chain = parallel_chain | prompt | llm | parser

In [29]:
main_chain.invoke('Can you summarize the video?')

'Demis Hassabis is interviewed on a podcast. He discusses the possibility of being an AI program created to interview people, and how it might affect his behavior. He also talks about his personal habits, such as his work routine and how it has changed over time. He then shifts the topic to explaining complex concepts in physics, suggesting that a more fundamental understanding of the world might be necessary to make progress in the field.'

In [30]:
import os
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# Load .env variables into environment
load_dotenv()

# Access the Groq key
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

VIDEO_ID = "Gfr50f6ZBvo"
QUESTION = (
    "Is the topic of nuclear fusion discussed in this video? "
    "If yes, what exactly was discussed?"
)

# --- YouTube transcript (new API) ---
try:
    ytt_api = YouTubeTranscriptApi()
    fetched = ytt_api.fetch(VIDEO_ID, languages=["en"])  # FetchedTranscript
    transcript_list = fetched.to_raw_data()              # [{text,start,duration}, ...]
    transcript = " ".join(x["text"] for x in transcript_list).strip()
except TranscriptsDisabled:
    print("No captions available for this video.")
    transcript = ""
except Exception as e:
    print(f"Failed to fetch transcript: {e}")
    transcript = ""

if not transcript:
    raise SystemExit("Cannot proceed: no transcript retrieved.")

# --- Chunking ---
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.create_documents([transcript])

# --- Local embeddings (FastEmbed uses onnxruntime CPU we pinned) ---
embeddings = FastEmbedEmbeddings()

# --- Vector store / retriever ---
vector_store = FAISS.from_documents(docs, embeddings)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

# --- Prompt & LLM (Groq) ---
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are a concise, helpful assistant. Use ONLY the context to answer.\n\n"
        "Context:\n{context}\n\n"
        "Question: {question}\n\n"
        "If the answer isn't in the context, say you can't find it."
    ),
)

# Options: "llama-3.1-8b-instant" (fast) or "llama-3.1-70b-versatile" (stronger)
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.2)

def format_docs(retrieved_docs):
    return "\n\n".join(doc.page_content for doc in retrieved_docs)

parallel = RunnableParallel({
    "context": retriever | RunnableLambda(format_docs),
    "question": RunnablePassthrough(),
})
parser = StrOutputParser()
main_chain = parallel | prompt | llm | parser

# --- Run ---
answer = main_chain.invoke(QUESTION)
print("\n=== Answer ===\n")
print(answer)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]


=== Answer ===

Yes, the topic of nuclear fusion is discussed in this video. 

The discussion revolves around the speaker's work on nuclear fusion, specifically:

1. Holding plasma in specific shapes for a record amount of time, which is a problem in fusion solved by the speaker's team.
2. Collaborating with EPFL in Switzerland to use their test reactor for research.
3. Applying AI methods to tackle the bottleneck problems in fusion, including using deep reinforcement learning to control high-temperature plasmas.
4. Discussing the potential of AI to accelerate and transform the field of fusion, which is a key area in energy and climate challenges.
